In [ ]:
import torch 
from torchinfo import summary 
 
model = torch.hub.load('pytorch/vision',  'resnet50', pretrained=True)
 
# 显示完整结构（含参数形状）
summary(model, input_size=(1, 3, 224, 224), depth=10)

In [ ]:
import timm 
model = timm.create_model('vit_large_patch16_224',  num_classes=1000)
 
# 冻结策略示例（保留最后3层为任务特定层）
for name, param in model.named_parameters(): 
    if not name.startswith(('blocks.21',  'blocks.22', 'blocks.23', 'head')):
        param.requires_grad  = False 
 
# 验证冻结效果 
print([n for n,p in model.named_parameters()  if p.requires_grad]) 
# 输出：['blocks.21...', 'blocks.22...', 'blocks.23...', 'head.weight',  'head.bias'] 


In [ ]:
freeze_ratio = 0.5  # 冻结50%的层 
total_layers = len(model.blocks) 
for i, block in enumerate(model.blocks): 
    if i < int(total_layers * freeze_ratio):
        for param in block.parameters(): 
            param.requires_grad  = False 

In [ ]:
def unfreeze_layers(epoch):
    if epoch == 5:
        for param in model.blocks[4:6].parameters():   # 第5个epoch解冻第5-6块 
            param.requires_grad  = True 

In [ ]:
# 分阶段解冻（示例调度器）
def unfreeze_scheduler(epoch):
    if epoch == 5:
        for name, param in model.named_parameters(): 
            if name.startswith('blocks.18'): 
                param.requires_grad  = True 
    elif epoch == 10:
        for name, param in model.named_parameters(): 
            if name.startswith('blocks.15'): 
                param.requires_grad  = True 

In [ ]:
# 遍历所有层 
for name, module in model.named_modules(): 
    if isinstance(module, (nn.Conv2d, nn.Linear)):
        print(f"\nLayer: {name}")
        print(f"Weight shape: {module.weight.shape}") 
        print(f"Bias shape: {module.bias.shape  if module.bias  is not None else 'None'}")
        print(f"Initial mean: {module.weight.data.mean():.4f}  ± {module.weight.data.std():.4f}") 

In [ ]:
import matplotlib.pyplot  as plt 
 
# 绘制某层权重直方图 
plt.hist(model.layer1[0].conv1.weight.data.cpu().numpy().flatten(),  bins=50)
plt.title("Layer1  Conv1 Weight Distribution")
plt.xlabel("Value") 
plt.ylabel("Frequency") 
plt.show() 

In [ ]:
# 检查初始化方法 
print(model._modules['conv1'].weight._initializer)  # 显示初始化函数 
print(model._modules['fc'].bias.initial_value)       # 查看bias初始值 

In [ ]:
from IPython.display  import display 
import pandas as pd 
 
params_data = []
for name, param in model.named_parameters(): 
    params_data.append({ 
        "Name": name,
        "Shape": str(tuple(param.shape)), 
        "Dtype": str(param.dtype), 
        "Requires_grad": param.requires_grad  
    })
    
display(pd.DataFrame(params_data))

In [ ]:
# 递归遍历工具
def print_structure(module, prefix=""):
    for name, child in module.named_children(): 
        print(f"{prefix}{name}: {type(child).__name__}")
        print_structure(child, prefix + "    ")
 
print_structure(model)

graph TD 
    A[Input] --> B[分支1]
    A --> C[分支2]
    B --> D[卷积层]
    D --> E[归一化层]
    C --> F[全连接层]
    E --> G[输出合并]
    F --> G 
    G --> H[Final Output]

In [ ]:
# # 安装可视化工具 
# pip install torchviz 
# from torchviz import make_dot 
 
# make_dot(model(torch.randn(1,3,224,224)),  
#          params=dict(model.named_parameters()), 
#          show_attrs=True).render("model", format="png")

In [ ]:
# 特性	named_parameters()	named_modules()
# 返回内容	所有可训练参数（Parameter对象）	所有模块实例（包含子模块）
# 遍历粒度	参数级（叶子节点）	模块级（树形结构）
# 典型用途	参数初始化/梯度裁剪	模型结构修改/层冻结
# 包含缓冲区(Buffer)	❌ 仅包含nn.Parameter	❌ 但可通过named_children()访问
# 递归控制	自动递归所有子模块	可通过depth参数控制递归深度

# 维度	named_modules()	named_parameters()
# 遍历对象	模块（如Conv2d、Linear）	参数（如weight、bias）
# 是否包含子模块	是（递归所有层级）	仅参数，不返回模块结构
# 是否包含不可训练参数	是（返回模块实例）	否（仅含requires_grad=True的参数）
# 典型用途	模型结构分析/层替换	参数初始化/梯度监控


In [ ]:
# 一、named_parameters()的参数来源
# 1. 参数注册时机
# 定义阶段：当继承nn.Module的类在__init__中调用self.param  = nn.Parameter()或使用nn.Linear等内置模块时，参数会自动注册
# 构建阶段：通过register_parameter(name, param)方法手动注册
# 加载预训练：load_state_dict()会将参数合并到命名空间中
# 2. 典型存储流程
# mermaid
# 复制
# sequenceDiagram 
#     participant User 
#     participant Module 
#     participant Parameter 
#     User->>Module: 定义nn.Linear(10,20)
#     Module->>Parameter: 创建weight/bias Parameter 
#     Parameter->>Module: 自动注册到_parameters字典 
#     User->>Module: 调用named_parameters()
#     Module->>User: 返回_parameters的键值对 
# 二、可能遗漏的参数类型
# 1. 非标准存储的参数
# 类型	示例	是否在named_parameters()中
# 手动张量	self.tensor  = torch.randn(10)	❌
# 缓冲区(Buffer)	self.register_buffer('running_mean',  torch.zeros(10))	❌（需用named_buffers()）
# 计算中间量	self.cache  = None	❌
# 非Parameter属性	self.scale  = 1.0	❌
# 2. 特殊模块例外
# 量化模型：伪量化参数可能存储在_qparams字典中
# 第三方扩展：如FairSeq的Adam优化器参数可能单独存储
# 三、参数完整性验证方法
# 1. 交叉检查工具
# python
# 复制
# def check_parameters(model):
#     # 检查所有可训练参数 
#     trainable_params = set(name for name, _ in model.named_parameters()) 
    
#     # 检查所有Parameter实例 
#     all_params = set()
#     for name, module in model.named_modules(): 
#         if hasattr(module, '_parameters'):
#             all_params.update(f"{name}.{k}"  for k in module._parameters.keys()) 
    
#     # 打印差异 
#     print("Missing in named_parameters():", all_params - trainable_params)
#     print("Extra in named_parameters():", trainable_params - all_params)
 
# check_parameters(model)